In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import os
from statsmodels.tsa.stattools import adfuller
# Matplotlib 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
# 처리할 CSV 파일 목록 (파일 경로 리스트)
file_list = [
    'C:/Users/acorn/Documents/python/일별소매가/단감_상품_10개_일별소매가_정제.csv',
    'C:/Users/acorn/Documents/python/일별소매가/단감_중품_10개_일별소매가_정제.csv'
]
# 결과 저장 디렉토리
output_dir = 'C:/Users/acorn/Documents/python/일별소매가/결과'
os.makedirs(output_dir, exist_ok=True)  # 디렉토리가 없으면 생성
# 각 파일 반복 처리
for file_path in file_list:
    # 파일 이름 추출
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    # 데이터 로드
    data = pd.read_csv(file_path, encoding='utf-8')
    # 쉼표 제거 처리 (숫자 변환)
    if data['평균'].dtype == 'object':
        data['평균'] = data['평균'].str.replace(',', '').astype(float)
    # 날짜 변환 및 데이터 준비
    data['구분'] = pd.to_datetime(data['구분'])
    data.set_index('구분', inplace=True)
    daily_prices = data['평균']
    # 주별 평균 데이터 생성
    weekly_prices = daily_prices.resample('W').mean()
    # **비어 있는 주 확인**
    missing_weeks = weekly_prices[weekly_prices.isna()].index
    # 결측 주 제외하고 SARIMA 학습 데이터 구성
    weekly_prices = weekly_prices.dropna()
    # SARIMA 모델 학습
    model = SARIMAX(weekly_prices, order=(5, 1, 0), seasonal_order=(1, 1, 1, 52))
    fitted_model = model.fit()
    # 1년(52주) 예측
    forecast = fitted_model.get_forecast(steps=52)
    forecast_values = forecast.predicted_mean
    # 예측 결과를 DataFrame으로 저장
    forecast_index = pd.date_range(start=weekly_prices.index[-1] + pd.DateOffset(weeks=1),
                                   periods=52, freq='W')
    forecast_df = pd.DataFrame({
        'Date': forecast_index,
        'Price': forecast_values.values
    })
    # **비어 있는 구간에 대한 예측 제거**
    forecast_df = forecast_df[~forecast_df['Date'].isin(missing_weeks)]
    # 과거 데이터와 예측 데이터 결합
    combined_df = pd.concat([
        pd.DataFrame({'Date': weekly_prices.index, 'Price': weekly_prices.values}),  # 과거 데이터
        forecast_df  # 유효 예측 데이터
    ])
    # CSV 파일로 저장
    output_file = os.path.join(output_dir, f'{file_name}_과거+예측데이터.csv')
    combined_df.to_csv(output_file, index=False, encoding='cp949')
    print(f"파일 저장 완료: {output_file}")
    # 시각화
    plt.figure(figsize=(14, 7))
    plt.plot(weekly_prices, label='Actual Data', marker='o')
    plt.plot(forecast_df['Date'], forecast_df['Price'], label='1-Year Forecast (Valid Weeks)', color='red', marker='x')
    plt.title(f'Weekly Price Prediction ({file_name})')
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.legend()
    plt.grid(True)
    plt.show()